<a href="https://colab.research.google.com/github/singh0021/TCD-Kaggle-Projects/blob/master/Kaggel_Linear_Regression_for_Income_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I have built various model and tried various approaches. This notebook is specific to final solution. If anyone interested to know other approaches and visualization techniques. Please refer to other #notebook in same directory with name : Machine Learning Solution

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

# Lets load both training and Testing Data

In [0]:
train_df =pd.read_csv("Data.csv")
train_df = train_df.drop(['Instance'], axis=1)
test_df = pd.read_csv("tcdml2019-20incomepredictiontest(without labels).csv")

# We need to exclude negative incomes if we are normalizing Income variable using log, else we will comment this part as we are getting good RMSE score by using scaler approach
#train_df = train_df[(train_df['Income in EUR'] > 0) ]

# Lets describe the training data and we see that there are negative incomes and few outliers. We can get rid of those rows

In [3]:
train_df.describe()
#train_df = train_df[(train_df['Income in EUR'] > 0)]
#train_df = train_df[train_df['Age'] < 110]
#train_df = train_df[train_df['Body Height [cm]'] < 225]



#Since accuracy improved a little bit after adding outliers, hence we kept outliers and let it run

,Year of Record,Age,Size of City,Wears Glasses,Body Height [cm],Income in EUR
count,111552.000000,111499.000000,1.119930e+05,111993.000000,111993.000000,1.119930e+05
mean,1999.421274,37.345304,8.388538e+05,0.500531,175.220192,1.092138e+05
std,11.576382,16.036694,2.196879e+06,0.500002,19.913889,1.498024e+05
min,1980.000000,14.000000,7.700000e+01,0.000000,94.000000,-5.696906e+03
25%,1989.000000,24.000000,7.273400e+04,0.000000,160.000000,3.077169e+04
50%,1999.000000,35.000000,5.060920e+05,1.000000,174.000000,5.733917e+04
75%,2009.000000,48.000000,1.184501e+06,1.000000,190.000000,1.260936e+05
max,2019.000000,115.000000,4.999251e+07,1.000000,265.000000,5.285252e+06


# Since there are 4-5 categorical variables and we have tested accucary of using few encoding such as one hot, label and target encoding. Target encoding gives me the best accurcay

In [0]:
# By doing smoothing we just trying to calculate weighted average so that data will not overfit if number of values used in 
# average is low. hence we used smooth factor of 10

def target_encoding(data, target, categories, smooth):
    
    train_target = data.copy()
    code_map = dict()   
    d_map = dict() # it will store global average of each variable
    
    for v in categories:
        p_mean = data[target].mean()
        n = data.groupby(v).size()
        u = data.groupby(v)[target].mean()
        u_smoothed = (n * u + smooth * p_mean) / (n + smooth)
        
        train_target.loc[:, v] = train_target[v].map(u_smoothed)        
        code_map[v] = u_smoothed
        d_map[v] = p_mean        
    return train_target, code_map, d_map

In [5]:
len(train_df['Profession'].unique())


1341

In [0]:
# Have used both target encoding on Training and testing dataset

categories = ['Country', 'Profession', 'University Degree','Gender']
train_smooth, target_m, d_m = target_encoding(train_df, 'Income in EUR', categories, 10)
test_t = test_df.copy()
for v in categories:
    test_t.loc[:, v] = test_t[v].map(target_m[v])

# We ran this model multiple times to see which features are adding value to the model. Hence we have selected below feature which gives best accuracy so far.

In [0]:
final_trhot = train_smooth[['Age','Year of Record','Body Height [cm]', 'Country', 'University Degree','Profession'
                                  ,'Size of City','Gender']]
test_col = test_t[['Age','Year of Record','Body Height [cm]', 'Country', 'University Degree','Profession',
                              'Size of City', 'Gender']]

In [0]:
# lets fill those missing values by taking means and since Income variable is skewed. We can take its log. I have commented log of Income variable as
# we shall get error on negative incomes and we can get similar RMSE by taking scaler approach itself

final_trhot = final_trhot.fillna(final_trhot.mean())
test_col = test_col.fillna(test_col.mean())


# To run this step, we need to take only positive value of Income Variable and hence data size will decrease. I have commented this step and not used in final prediction
#y_t = np.log(train_df['Income in EUR'])

In [0]:
# Initially i have used linear regression model which was giving me good accuracy. However, i have tried various models 
# and got best RMSE in Random Forest Regressor and Hence i have commented linear regression.

#from sklearn.preprocessing import StandardScaler
# Xhot_tr, Xhot_ts, Yhot_tr, Yhot_ts = train_test_split(final_trhot, train_df['Income in EUR'], test_size=0.25, random_state = 7)

#scaler = StandardScaler().fit(Xhot_tr)
#rescaled_X_train = scaler.transform(Xhot_tr)

#lm.fit(rescaled_X_train,Yhot_tr)

#rescaled_X_ts = scaler.transform(Xhot_ts)
#predictions = lm.predict(rescaled_X_ts)
#print('The accuracy of the Linear Regression is',r2_score(Yhot_ts,predictions))
#print ('MSE is: \n', np.sqrt(mean_squared_error(Yhot_ts,predictions)))


In [9]:
## Lets build RandomForest Regression model
# estimators were chosen by running model multiple times by verifying accurcay and RMSE, hence estimators chosen were 250.

import numpy
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

Xhot_tr, Xhot_ts, Yhot_tr, Yhot_ts = train_test_split(final_trhot, train_df['Income in EUR'], test_size=0.25, random_state = 7)

cls = RandomForestRegressor(n_estimators=250, criterion='mse', max_depth=30)


scaler = StandardScaler().fit(Xhot_tr)
rescaled_X_train = scaler.transform(Xhot_tr)

rescaled_X_ts = scaler.transform(Xhot_ts)

cls.fit(rescaled_X_train,Yhot_tr)
y_pred = cls.predict(rescaled_X_ts)

print('The accuracy of the Random Regression is',r2_score(Yhot_ts,y_pred))
print ('RMSE is: \n', np.sqrt(mean_squared_error(Yhot_ts,y_pred)))


The accuracy of the Random Regression is 0.8243782866619312
RMSE is: 
 61571.487569968805


# It is also important to note that RMSE score decreases if we take log of Income variable on validation data.Also, if we are not taking log of Income variable, we need to scale our data. Result are pretty much same on testing data only RMSE differs on local validation data

In [0]:
# After I got good accuracy on Random Forest Regression, I have tried other approaches such as 
#XGB boost, GBM, CatBoost and ensemble one. Please do check conclusion of this notebook.

In [0]:
# There was error in column name due to spacing in headers, hence we fixed it using regex expressions

import re

regex = re.compile(r"\[|\]|<", re.IGNORECASE)

final_trhot.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in final_trhot.columns.values]
#Xhot_ts.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in Xhot_ts.columns.values]
test_col.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in test_col.columns.values]

In [12]:
# We tried implemented to improve accuracy more by using XGBoost. Estimators were chosen by running model multiple times by checking accurcay and RMSE

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
Xhot_tr, Xhot_ts, Yhot_tr, Yhot_ts = train_test_split(final_trhot, train_df['Income in EUR'], test_size=0.25, random_state = 7)

scaler = StandardScaler().fit(Xhot_tr)
rescaled_X_train = scaler.transform(Xhot_tr)

rescaled_X_ts = scaler.transform(Xhot_ts)

my_model = XGBRegressor(n_estimators=1000)
xgb = my_model.fit(rescaled_X_train, Yhot_tr, early_stopping_rounds=5, 
             eval_set=[(rescaled_X_ts, Yhot_ts)], verbose=False)

y_pred = xgb.predict(rescaled_X_ts)

print('The accuracy of the XGB Regression is',r2_score(Yhot_ts,y_pred))
print ('RMSE is: \n', np.sqrt(mean_squared_error(Yhot_ts,y_pred)))


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[08:25:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The accuracy of the XGB Regression is 0.8388044065504278
RMSE is: 
 58988.46894758571


In [13]:
len(rescaled_X_ts)

27999

In [14]:
# Since XGBoost improved our RMSE on test data. I tried to use GBM too and it gave me a little boost. Estimators were decided by running model multiple times
# and best RMSE estimator was chosen.


from sklearn import ensemble

clf = ensemble.GradientBoostingRegressor(n_estimators = 1200, max_depth = 7, min_samples_split = 2,
          learning_rate = 0.1, loss = 'ls')

clf.fit(rescaled_X_train, Yhot_tr)

rescaled_X_ts = scaler.transform(Xhot_ts)
predictions = clf.predict(rescaled_X_ts)

print('The accuracy of the XGB Regression is',r2_score(Yhot_ts,predictions))
print (np.sqrt(mean_squared_error(Yhot_ts, predictions)))

The accuracy of the XGB Regression is 0.8387034413641276
59006.93982919416


In [15]:
#Lets install catboost
!pip install catboost

     |████████████████████████████████| 62.7MB 468kB/s 


In [16]:
# Lets try on CatBoost also. Hyper parameters we have choosen by running model multiple times to see if RMSE improves

from catboost import Pool, CatBoostRegressor

cb = CatBoostRegressor(learning_rate=0.6, eval_metric='RMSE')
cb.fit(rescaled_X_train, Yhot_tr)

pred_cb = cb.predict(rescaled_X_ts)
print('The accuracy of the CAT Regression is',r2_score(Yhot_ts,pred_cb))
print (np.sqrt(mean_squared_error(Yhot_ts, pred_cb)))

0:	learn: 97680.5366678	total: 65.1ms	remaining: 1m 5s
1:	learn: 79282.2746772	total: 81.7ms	remaining: 40.8s
2:	learn: 71694.3373939	total: 96.9ms	remaining: 32.2s
3:	learn: 68244.8481124	total: 112ms	remaining: 27.8s
4:	learn: 66387.8317059	total: 126ms	remaining: 25.2s
5:	learn: 65380.5608201	total: 140ms	remaining: 23.3s
6:	learn: 64708.7349565	total: 155ms	remaining: 22s
7:	learn: 63976.4922164	total: 169ms	remaining: 21s
8:	learn: 63552.8650642	total: 184ms	remaining: 20.2s
9:	learn: 62901.8847681	total: 199ms	remaining: 19.7s
10:	learn: 62193.2999011	total: 215ms	remaining: 19.3s
11:	learn: 61910.4331420	total: 229ms	remaining: 18.9s
12:	learn: 61407.1588230	total: 245ms	remaining: 18.6s
13:	learn: 61009.3606543	total: 259ms	remaining: 18.2s
14:	learn: 60774.4366280	total: 279ms	remaining: 18.3s
15:	learn: 60546.0468672	total: 294ms	remaining: 18.1s
16:	learn: 60401.8525980	total: 310ms	remaining: 17.9s
17:	learn: 60012.2496447	total: 325ms	remaining: 17.7s
18:	learn: 59715.3365

In [0]:
#Lets do prediction on testing data. We need to scale test data as well
# We did prediction on other algorithms and I have commented as finally i have used RandomForest regression model

#test_X_ts = scaler.transform(test_col)

#pred0 = xgb.predict(test_X_ts)
#pred1 = clf.predict(test_X_ts)
#pred2 = cb.predict(test_X_ts)
#pred3 = lb.predict(test_X_ts)
#actual_predictions = (pred0+pred1+pred2+pred3)/4


In [0]:
# Lets predict Income variable for test data using Random Forest Regression

test_X_ts = scaler.transform(test_col)
predictions = cls.predict(test_X_ts)


#  Below predcitions were used when we take log of Income variable but it didn't help RMSE on public board, hence i got rid of below method.

#pred = cls.predict(test_col)
#predictions = np.exp(pred)

In [18]:
predictions.max()

3543971.0548611972

In [0]:
#lets submit our predictions on test data


submissions =pd.DataFrame({'Instance': test_df['Instance'], 'Income': predictions}, columns =['Instance','Income'])
filename ='tcd ml 2019-20 income prediction submission file.csv'
submissions.to_csv(filename, index=False)
print('Saved File '+ filename)

Saved File tcd ml 2019-20 income prediction submission file.csv


In [0]:
# In conclusion, many regression techniques gave me better RMSE on validation data and other boosting algorithm gave me boost to RMSE; however model got overfitted on testing data and brought my RMSE down.